In [113]:
import pymongo
from pymongo import MongoClient, GEO2D
from bson.son import SON
client = MongoClient('mongodb://localhost:27017/')
db = client.restaurants
business = db.business
business.find_one()
business.aggregate([{ "$addFields": { "geolocation": ["$longitude", "$latitude" ] }},{ "$out": "business" }])
business.find_one()
business.create_index([("geolocation", GEO2D)])
result = db.business.find({ "$and":[{ "city": "Philadelphia" },{"categories":"Restaurants"},{"geolocation":SON([("$near", [-75.0447831, 40.0261449]),("$maxDistance", 0.2)])}]}).sort([{"stars",pymongo.DESCENDING},{"review_count",pymongo.DESCENDING}]).limit(10)
query = list(result)
df = pd.DataFrame(query)
keyword_df = df.get(['name', 'stars', 'review_count', 'latitude', 'longitude'])
keyword_df

,name,stars,review_count,latitude,longitude
0,Cheers Cafe,5.0,6,39.986842,-75.101025
1,Gayle,4.5,18,39.940934,-75.147618
2,Yummy Diner,4.5,6,39.961100,-75.224897
3,Shank & Evelyn's Frankies At Night Eclipse Cafe,4.5,5,39.938326,-75.159773
4,Saute,4.0,25,39.937879,-75.144534
5,Bistro Juliana,4.0,14,39.975071,-75.118352
6,Roselena's,4.0,10,39.929374,-75.163843
7,Astral Plane,4.0,9,39.945161,-75.170498
8,El Provocon Restaurant II,4.0,7,40.022009,-75.126147
9,Aspen,4.0,7,39.968780,-75.178154


In [114]:
import folium
import pandas as pd     
from IPython.display import display
PA_coordinates = [39.9525839, -75.1652215]
restaurant_map = folium.Map(location = PA_coordinates, zoom_start = 12) 
for i in range(0,len(keyword_df)):
    folium.Marker(location=[keyword_df.iloc[i]['latitude'], keyword_df.iloc[i]['longitude']],
                  popup=keyword_df.iloc[i]['name'] + '\n' + \
                        'Rating: ' + str(keyword_df.iloc[i]['stars']) + '\n' + \
                        'Reviews: ' + str(keyword_df.iloc[i]['review_count']), \
                  icon=folium.Icon(color='red')).add_to(restaurant_map)
restaurant_map